"""

Constraints:
Each clothing item is given to exactly one child
Clothing size must match the chosen child’s size
Each child receives at least one summer item and one winter item


Objective:
◦ Let R = the sum of the retail prices for all of the clothing
◦ Let N = the number of children
◦ R/N = the fair share for each child
◦ Let Ci = the sum of the clothing prices for child i
◦ | R/N - Ci | = the amount over/under that child i received
◦ Your project code must compute a distribution that will provide the absolute minimum value for
D, where D is:


Signed Academic Integrity statement must be submitted


Your goal is to distribute them as fairly as possible, while ensuring that the clothes fit and that
each child gets at least one summer item and one winter item.
"""

In [1]:
fname = "example_input/ex6_2children_6clothes.txt"

#### 1. Load and Prepare data

In [2]:
with open(fname) as f:
    flines = f.readlines()
    clothes_records  = []
    children_records = []
    for fline in flines:
        if 'Clothes' in fline or 'Children' in fline:
            continue
        else:
            if fline[0] == 'A':
                cloth_record = fline.split("\t")
                clothes_records.append({
                                               'cloth_id': cloth_record[0],
                                               'cloth_size': cloth_record[1],
                                               'cloth_season': cloth_record[2],
                                               'cloth_price': cloth_record[3].strip()
                                       }
                                      )
            else:
                children_record = fline.split("\t")
                children_records.append(
                    {
                        'child_id': children_record[0],
                        'child_size': children_record[1].strip(),
                        'summer_clothes': [],
                        'winter_clothes': []
                    }
                )

In [3]:
import pandas as pd

In [4]:
pd.DataFrame(clothes_records)

,cloth_id,cloth_size,cloth_season,cloth_price
0,A1,S,summer,24
1,A2,ALL,winter,20
2,A3,ALL,winter,30
3,A4,XL,summer,15
4,A5,XL,summer,36
5,A6,S,summer,26


In [5]:
pd.DataFrame(children_records)

,child_id,child_size,summer_clothes,winter_clothes
0,C1,XL,[],[]
1,C2,S,[],[]


#### 2. Compute all static variables before starting the optimization algorithm

In [6]:
N = len(children_records) # the number of children
N

2

In [7]:
# R = the sum of the retail prices for all of the clothing
R = 0
for cloth_record in clothes_records:
    R += float(cloth_record['cloth_price'])
R

151.0

In [8]:
## R/N = the fair share for each child
r_by_n_ratio = R/N
r_by_n_ratio

75.5

#### 3. Matrix preparation

In [9]:
len(clothes_records), len(children_records)

(6, 2)

In [10]:
child_ids = [f"{each['child_id']}_{each['child_size']}" for each in children_records]
child_ids

['C1_XL', 'C2_S']

In [11]:
summer_cloth_ids = [f'SU_{each["cloth_id"]}_{each["cloth_size"]}_{each["cloth_price"]}' for each in clothes_records if each['cloth_season']=='summer']
summer_cloth_ids

['SU_A1_S_24', 'SU_A4_XL_15', 'SU_A5_XL_36', 'SU_A6_S_26']

In [12]:
winter_cloth_ids = [f'WI_{each["cloth_id"]}_{each["cloth_size"]}_{each["cloth_price"]}' for each in clothes_records if each['cloth_season']=='winter']
winter_cloth_ids

['WI_A2_ALL_20', 'WI_A3_ALL_30']

In [13]:
summer_df = pd.DataFrame(index=child_ids, columns=summer_cloth_ids)
summer_df

,SU_A1_S_24,SU_A4_XL_15,SU_A5_XL_36,SU_A6_S_26
C1_XL,NaN,NaN,NaN,NaN
C2_S,NaN,NaN,NaN,NaN


In [14]:
winter_df = pd.DataFrame(index=child_ids, columns=winter_cloth_ids)
winter_df

,WI_A2_ALL_20,WI_A3_ALL_30
C1_XL,NaN,NaN
C2_S,NaN,NaN


In [15]:
summer_df_i, summer_df_j = summer_df.shape
summer_df_i, summer_df_j

(2, 4)

In [16]:
summer_df_cols = summer_df.columns.tolist()
print(summer_df_cols)
summer_df_indices = summer_df.index.tolist()
print(summer_df_indices)

['SU_A1_S_24', 'SU_A4_XL_15', 'SU_A5_XL_36', 'SU_A6_S_26']
['C1_XL', 'C2_S']


In [17]:
for i in range(summer_df_i):
    for j in range(summer_df_j):        
        cl_size = summer_df_cols[j].split("_")[2]
        cl_price = summer_df_cols[j].split("_")[3]
        cd_size = summer_df_indices[i].split("_")[1]
        if cl_size == cd_size or cl_size == "ALL":
            # assign this cloth to the child
            summer_df.loc[summer_df_indices[i], summer_df_cols[j]] = cl_price
                
            

In [18]:
summer_df

,SU_A1_S_24,SU_A4_XL_15,SU_A5_XL_36,SU_A6_S_26
C1_XL,NaN,15,36,NaN
C2_S,24,NaN,NaN,26


In [19]:
winter_df_i, winter_df_j = winter_df.shape
winter_df_i, winter_df_j

(2, 2)

In [20]:
winter_df_cols = winter_df.columns.tolist()
print(winter_df_cols)
winter_df_indices = winter_df.index.tolist()
print(winter_df_indices)

['WI_A2_ALL_20', 'WI_A3_ALL_30']
['C1_XL', 'C2_S']


In [21]:
for i in range(winter_df_i):
    for j in range(winter_df_j):
        cl_size = winter_df_cols[j].split("_")[2]
        cl_price = winter_df_cols[j].split("_")[3]
        cd_size = winter_df_indices[i].split("_")[1]
        if cl_size == cd_size or cl_size == "ALL":
            # assign this cloth to the child
            winter_df.loc[winter_df_indices[i], winter_df_cols[j]] = cl_price

In [22]:
summer_df

,SU_A1_S_24,SU_A4_XL_15,SU_A5_XL_36,SU_A6_S_26
C1_XL,NaN,15,36,NaN
C2_S,24,NaN,NaN,26


In [23]:
winter_df

,WI_A2_ALL_20,WI_A3_ALL_30
C1_XL,20,30
C2_S,20,30


In [24]:
matrix_df = summer_df.join(winter_df)
matrix_df

,SU_A1_S_24,SU_A4_XL_15,SU_A5_XL_36,SU_A6_S_26,WI_A2_ALL_20,WI_A3_ALL_30
C1_XL,NaN,15,36,NaN,20,30
C2_S,24,NaN,NaN,26,20,30


#### 4. Algorithm

In [25]:
import numpy as np

In [26]:
matrix_df = matrix_df.apply(pd.to_numeric)
matrix_df

,SU_A1_S_24,SU_A4_XL_15,SU_A5_XL_36,SU_A6_S_26,WI_A2_ALL_20,WI_A3_ALL_30
C1_XL,NaN,15.0,36.0,NaN,20,30
C2_S,24.0,NaN,NaN,26.0,20,30


In [27]:
matrix_df.to_numpy()

array([[nan, 15., 36., nan, 20., 30.],
       [24., nan, nan, 26., 20., 30.]])

In [28]:
matrix_df_with_fair_cost = matrix_df - r_by_n_ratio
matrix_df_with_fair_cost # let us get back to this

,SU_A1_S_24,SU_A4_XL_15,SU_A5_XL_36,SU_A6_S_26,WI_A2_ALL_20,WI_A3_ALL_30
C1_XL,NaN,-60.5,-39.5,NaN,-55.5,-45.5
C2_S,-51.5,NaN,NaN,-49.5,-55.5,-45.5


In [29]:
matrix_df.fillna(-1, inplace=True)
matrix_df

,SU_A1_S_24,SU_A4_XL_15,SU_A5_XL_36,SU_A6_S_26,WI_A2_ALL_20,WI_A3_ALL_30
C1_XL,-1.0,15.0,36.0,-1.0,20,30
C2_S,24.0,-1.0,-1.0,26.0,20,30


In [30]:
def create_and_get_path():
    path = {}
    for child_record in children_records:
        path[child_record['child_id']] = {'summer_clothes': [], 'winter_clothes': [], 'ci': 0,
                                         'summer_winter_constraint': False}
    return path

In [31]:
matrix_df_i, matrix_df_j = matrix_df.shape
matrix_df_i, matrix_df_j 

(2, 6)

In [32]:
matrix_df_indices = matrix_df.index.tolist()
matrix_df_cols = matrix_df.columns.tolist()

In [33]:
matrix_df_indices

['C1_XL', 'C2_S']

In [34]:
matrix_df_cols

['SU_A1_S_24',
 'SU_A4_XL_15',
 'SU_A5_XL_36',
 'SU_A6_S_26',
 'WI_A2_ALL_20',
 'WI_A3_ALL_30']

In [35]:
matrix_df.loc[matrix_df_indices[1],matrix_df_cols[3]]

26.0

In [36]:
matrix_df

,SU_A1_S_24,SU_A4_XL_15,SU_A5_XL_36,SU_A6_S_26,WI_A2_ALL_20,WI_A3_ALL_30
C1_XL,-1.0,15.0,36.0,-1.0,20,30
C2_S,24.0,-1.0,-1.0,26.0,20,30


In [37]:
matrix_df = matrix_df.fillna(-1)

In [38]:
matrix_df

,SU_A1_S_24,SU_A4_XL_15,SU_A5_XL_36,SU_A6_S_26,WI_A2_ALL_20,WI_A3_ALL_30
C1_XL,-1.0,15.0,36.0,-1.0,20,30
C2_S,24.0,-1.0,-1.0,26.0,20,30


In [41]:
import copy
class Executor:
    
    def __init__(self):
        self.optimal_min = 99999999
    def print_all_paths_helper(self, matrix_df, i, j, m, n, structure_of_path, seen):
        #print(i, j)
        if i < 0:
            return
        if i > m - 1:
            #print("YES")
            return

        if j > n - 1:
            seen_str = ""
            for each in structure_of_path:
                seen_str += f"{each[0]}{each[1]}"
            if seen_str not in seen:
                print(structure_of_path)
                D = 0
                for cd_ix in range(matrix_df_i):
                    c_i = 0
                    for (pi, pj) in structure_of_path:
                        if pi == cd_ix:
                            
                            c_i += matrix_df.loc[matrix_df_indices[pi],
                                                               matrix_df_cols[pj]]
                                
                    D += abs(r_by_n_ratio - c_i)
                seen.append(seen_str)
                print("D: ", D)
                if D < self.optimal_min:
                    self.optimal_min = D

            return
        if j < 0:
            return
        if matrix_df.loc[matrix_df_indices[i],matrix_df_cols[j]] < 0:
            # not eligible
            return
        new_path_struct = copy.deepcopy(structure_of_path) # [0,0]
        new_path_struct.append((i, j))
        # i exploration
        for ix in range(matrix_df_i):
            ### MAIN RECURSION HAPPENS HERE
            self.print_all_paths_helper(matrix_df, i + ix, j + 1 , m, n, new_path_struct, seen)
            self.print_all_paths_helper(matrix_df, i - ix - 1, j + 1 , m, n, new_path_struct, seen)


        new_path_struct = []

    def print_all_paths(self, matrix_df, matrix_df_i, matrix_df_j):
        for idx in range(matrix_df_i):
            path_struct = []
            seen = []
            self.print_all_paths_helper(matrix_df, idx, 0, matrix_df_i, matrix_df_j, path_struct, seen)
        print(self.optimal_min)
exec_obj = Executor()
exec_obj.print_all_paths(matrix_df, matrix_df_i, matrix_df_j)
print("Optimal Minimum: " , exec_obj.optimal_min)

[(1, 0), (0, 1), (0, 2), (1, 3), (1, 4), (1, 5)]
D:  49.0
[(1, 0), (0, 1), (0, 2), (1, 3), (1, 4), (0, 5)]
D:  11.0
[(1, 0), (0, 1), (0, 2), (1, 3), (0, 4), (0, 5)]
D:  51.0
[(1, 0), (0, 1), (0, 2), (1, 3), (0, 4), (1, 5)]
D:  9.0
9.0
Optimal Minimum:  9.0


In [40]:
matrix_df

,SU_A1_S_24,SU_A4_XL_15,SU_A5_XL_36,SU_A6_S_26,WI_A2_ALL_20,WI_A3_ALL_30
C1_XL,-1.0,15.0,36.0,-1.0,20,30
C2_S,24.0,-1.0,-1.0,26.0,20,30
